<a href="https://colab.research.google.com/github/RayThibodeaux/password-strength/blob/main/Password_Project_TF_IDF_Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Dataframes, manipulation, and widgets
import time
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import IntProgress
from ipywidgets import interact, interactive, fixed, interact_manual

#Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Balance data
from sklearn.utils import class_weight

#Train test split
from sklearn.model_selection import train_test_split

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

#Cross Validation
from sklearn.model_selection import cross_val_score

#Evaluate
from sklearn.metrics import confusion_matrix,accuracy_score

In [3]:
#Creating a dataframe with the password data
passwords = pd.read_csv('passwords.csv',on_bad_lines='skip')
passwords

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1
...,...,...
669635,10redtux10,1
669636,infrared1,1
669637,184520socram,1
669638,marken22a,1


In [4]:
#View na values in data
passwords.isna().sum()

password    1
strength    0
dtype: int64

In [5]:
#View na values in data
passwords[passwords['password'].isna()]

,password,strength
367579,NaN,0


In [6]:
#Remove na value since it's only one value out of 669k
passwords = passwords.dropna()

In [7]:
#Passsword strength distribution - Normal and Discrete
fig = px.histogram(passwords, x="strength",nbins=5,text_auto=True)
fig.update_layout(bargap=0.5)
fig.show()

In [8]:
#Creating an array of data - we need an array because it is more efficient
password_a = np.array(passwords)

#Password array
indep_x = [i[0] for i in password_a]
indep_x = np.array(indep_x)

#Strength array
depend_y = [i[1] for i in password_a]
depend_y = np.array(depend_y)

In [9]:
#Balancing the weight of each class for 
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(depend_y),y=depend_y)
print(class_weights)

[2.4884115  0.44930063 2.68488158]


In [10]:
#Function to split each word to character
def split_text(dataset):
    character=[]
    for i in dataset:
        character.append(i)
    return character

In [11]:
#Tokenize each character and transform to vector - tfidf
tfidf_vector = TfidfVectorizer(tokenizer=split_text)
tfidf_word_vector = tfidf_vector.fit_transform(indep_x)

In [12]:
#View tfidf vector
tfidf_view = tfidf_word_vector[0]
tfidf_df = pd.DataFrame(tfidf_view.T.todense(), index=tfidf_vector.get_feature_names_out(), columns=['TF-IDF'])
tfidf_df.sort_values(by=['TF-IDF'], ascending=False)

,TF-IDF
7,0.591303
5,0.566899
z,0.335926
k,0.292247
d,0.285631
...,...
\,0.000000
],0.000000
^,0.000000
_,0.000000


In [13]:
#Splitting dataset into train test split for tfidf - %80 train, %20 test
X_train, X_test, y_train, y_test = train_test_split(tfidf_word_vector, depend_y, test_size=0.2)

In [14]:
#View the balanced weights for each class
c_weights = class_weight.compute_class_weight('balanced', classes=np.unique(depend_y),y=depend_y)
print(c_weights)

[2.4884115  0.44930063 2.68488158]


In [15]:
#Creating model parameters
log_reg_params = [{'class_weight': 'balanced',"max_iter": 10000}]
dec_tree_params = [{'class_weight': 'balanced'}]
rand_for_params = [{'class_weight': 'balanced'}]
l_svc_params = [{'class_weight': 'balanced'}]

In [16]:
#Creating list of models
model_list = [
    ["Logistic Regression", LogisticRegression, log_reg_params],
    ["Decision Tree", DecisionTreeClassifier, dec_tree_params],
    ["Random Forest", RandomForestClassifier, rand_for_params],
    ["SVC",LinearSVC, l_svc_params]
]

In [17]:
#Iterate through each model to train,test,and score
overview = []
for mn, m, p_list in model_list:
    for p in p_list:
        model = m(**p)
        model.fit(X_train,y_train)
        score = model.score(X_test,y_test)
        overview.append((mn,m,p,score))

In [18]:
#View scores of each model and sort by most accurate
overview.sort(key=lambda x:x[-1], reverse=True)
for mn, m, p, score in overview:
    print(mn, p, score)

Random Forest {'class_weight': 'balanced'} 0.947135774445971
Decision Tree {'class_weight': 'balanced'} 0.9166417776715847
SVC {'class_weight': 'balanced'} 0.7890508332835553
Logistic Regression {'class_weight': 'balanced', 'max_iter': 10000} 0.7295860462337973


In [20]:
#Iterate through each model to train,test,and score - TFIDF Vector - Cross Validation
overview = []
for mn, m, p_list in model_list:
    for p in p_list:
        model = m(**p)
        cross_score = cross_val_score(model,tfidf_word_vector,depend_y,cv=5,scoring='accuracy').mean()
        overview.append((mn,m,p,cross_score))

In [21]:
#View scores of each model and sort by most accurate - Cross Validation
overview.sort(key=lambda x:x[-1], reverse=True)
for mn, m, p, cross_score in overview:
    print(mn, p, cross_score)

Random Forest {'class_weight': 'balanced'} 0.9469729217743594
Decision Tree {'class_weight': 'balanced'} 0.9166670421043726
SVC {'class_weight': 'balanced'} 0.7889176115862193
Logistic Regression {'class_weight': 'balanced', 'max_iter': 10000} 0.729161533209916


In [ ]:
#Random Forest Model - TFIDF Vector
ran_model = RandomForestClassifier(class_weight='balanced')
ran_model.fit(X_train,y_train)

In [ ]:
#Interactive password input that predicts password strength
#Widgets
password_text = widgets.Text(
    placeholder='Enter Password',
    description='Password:',
    disabled=False
)
button = widgets.Button(description='Submit Password')
out = widgets.Output()

In [ ]:
#Button function
def on_button_clicked(_):
      with out:
          #Button action
          out.clear_output()
          X_manual = tfidf_vector.transform([password_text.value])
          pr = ran_model.predict(X_manual)
          #Progress bar
          prog = widgets.IntProgress(min=0,max=2,style={'bar_color': 'lightblue'})
          prog.value = pr[0]
          display(prog)
          #Simple Bruteforce and Dictionary attack prevention
          button.disabled = True
          time.sleep(3)
          button.disabled = False
#Linking button and function
button.on_click(on_button_clicked)
# Display widgets
widgets.VBox([password_text,button,out])